In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import math
import random
import numpy as np
import gymnasium as gym
from robot_model import *
torch.manual_seed(42)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# # if GPU is to be used
# device = torch.device(
#     "cuda" if torch.cuda.is_available() else
#     "mps" if torch.backends.mps.is_available() else
#     "cpu"
# )
device = 'cpu'

In [2]:
from gymnasium import spaces

class CDPR4_DQN_env(CDPR4):
    def __init__(self, start_state=np.array([.0, .0, 1.0, .0, .0, .0]), desired_state=np.array([.0, .0, 2.0, .0, .0, .0]), pos=np.array([.0, .0, 1.0]), params=params, approx=1, mass=1):
        super().__init__(pos=np.array([.0, .0, 1.]), params=params, approx=1, mass=1)

        self.start_state = start_state.copy()  # start position 1m on Z
        self.cur_state = np.array([.0, .0, 1., .0, .0, .0]) # X=[pos, vel] in control
        self.reward = 0 # reward 0 is 0 error in position, closer position to desired -> higher reward 
        self.desired_state = desired_state
        self.v = np.array([.0, .0, .0])
    
        self.max_speed = 10
        self.max_force = 45 
        
        self.action_space = spaces.MultiDiscrete(np.array([[self.max_force], [self.max_force], [self.max_force], [self.max_force]]))
        self.observation_space = spaces.Box(low=np.array([-1.154, -1.404, .0, -self.max_speed, -self.max_speed, -self.max_speed]), 
                                            high=np.array([1.154, 1.404, 3.220,  self.max_speed, self.max_speed, self.max_speed]))
        self._max_episode_steps = 500 # default in gymnasium env is 1000
        self._elapsed_steps = 0
        self.dt = 0.5 # seconds between state update
        
    def reset(self):
        state = self.observation_space.sample()
        
        # self.reward = -1 # 1 meter away from desired position
        self._elapsed_steps = 0
        
        self.pos = state[:3].flatten()
        self.v = state[3:].flatten()
        self.control = np.array([.0, .0, .0, .0])
        
        self.reward = np.linalg.norm(self.pos - self.desired_state[:3].flatten()).astype(np.float32)
        
        return state, {}
    
    def step(self, action):
        pos = self.cur_state[:3].flatten()
        vel = self.cur_state[3:].flatten()
        m = self.m

        dt = self.dt
        costs = 1e-3*np.linalg.norm(pos - self.desired_state[:3].flatten()).astype(np.float32) # reward function includes only position, no velocities
        # TODO: add velocity to cost
        # costs += 5e-3*np.linalg.norm(vel - self.desired_state[3:].flatten()) # velocity factor
        # print(f'cost {costs}')
        
        u = action.cpu().numpy().reshape((4,1))
        dXdt = self.B() @ u + np.array([.0, .0, .0, .0, .0, -g]).reshape((6,1))
        # print(f'u {u.shape}')
        # print(f'B {self.B().shape}')
        # print(f'dXdt {dXdt.shape}')
       
        new_vel = vel + dXdt[:3].flatten()*dt
        new_pos = pos + vel*dt + 0.5*dXdt[3:].flatten()*dt**2

        self.pos = new_pos
        self.v = new_vel

        state = np.hstack((new_pos, new_vel))
        self.cur_state = state
        
        terminated = np.allclose(self.cur_state, self.desired_state, atol=1e-03) # reached desired position
        
        self._elapsed_steps += 1

        truncated = False
        if self._elapsed_steps >= self._max_episode_steps:
            truncated = True
        
        return state, -costs, terminated, truncated, {} #
        

In [3]:
env = CDPR4_DQN_env()

/Users/damirnurtdinov/miniconda3/envs/cdpr_mujoco/lib/python3.9/site-packages/gymnasium/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions*4) # 4 actuators

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x.view(-1, 4, 45) # (batch size, 4 actuators, 45 probabilities)

In [6]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = int(env.max_force) #from 0 to 44
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            # print(f'policy_net(state) {torch.max(policy_net(state).reshape((4,45)), 1).indices.view(4, 1)}')
            # print(f'policy_net(state) {policy_net(state).shape}')
            return torch.max(policy_net(state).reshape((4,45)), 1).indices.view(4, 1)
    else:
        # return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)
        return torch.from_numpy(env.action_space.sample())

episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward).reshape((BATCH_SIZE,1))
    reward_batch = reward_batch@torch.tensor([[1,1,1,1]], dtype=torch.float32) # (128, 4)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    # print(f'action_batch {action_batch.shape}') # 128, 1, 4, 1
    # print(f'state_batch {state_batch.shape}') # [128, 6]
    # print(f'policy_net(state_batch) {policy_net(state_batch).shape}') # (128, 4, 45)
    policy_output = policy_net(state_batch)
    act_vals = torch.zeros((BATCH_SIZE, 4, 1))
    for i in range(BATCH_SIZE):
        act = torch.max(policy_output[i].reshape((4,45)), 1).indices.view(4, 1).float()
        act_vals[i] = act.clone().detach().requires_grad_(True)
    # print(act_vals)
    state_action_values = torch.tensor(act_vals, requires_grad=True)
    # print(f"state_action_values {state_action_values.shape}")
    # print(f"non_final_mask {non_final_mask.shape}")
    # print(f"non_final_next_states {non_final_next_states.shape}")
    # state_action_values = policy_net(state_batch).gather(1, action_batch.reshape((128, 4, 1)))

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros((BATCH_SIZE, 4, 1), device=device, requires_grad=True)
    with torch.no_grad():
        # print(f'target_net(non_final_next_states) {target_net(non_final_next_states).shape}')
        # print(f'')
        next_state_values[non_final_mask] = torch.max(target_net(non_final_next_states).reshape((BATCH_SIZE, 4,45)), dim=2).values.view(BATCH_SIZE, 4, 1) # target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch.reshape((BATCH_SIZE, 4, 1))
    # print(f'next_state_values {next_state_values}')

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    # print(f'state_action_values {state_action_values.view(BATCH_SIZE, -1)}')
    # print(f'expected_state_action_values {expected_state_action_values.view(BATCH_SIZE, -1)}')
    # loss = criterion(state_action_values.reshape([BATCH_SIZE, 1, 4, 1]), expected_state_action_values.unsqueeze(1))
    loss = criterion(state_action_values.view(BATCH_SIZE, -1), expected_state_action_values.view(BATCH_SIZE, -1))  # Flatten both for loss calculation

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100.0)
    optimizer.step()

In [8]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state).reshape((1, 1, 4,1))
        # print(f'action chosen: {action.shape}')
        observation, reward, terminated, truncated, _ = env.step(action)
        reward = torch.tensor([reward.astype(np.float32)], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

/var/folders/6j/s19nhfjj4pv14vny7nrq8lf00000gn/T/ipykernel_86962/4193878755.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_action_values = torch.tensor(act_vals, requires_grad=True)


RuntimeError: Expected !nested_tensorlist[0].empty() to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)